# Qalam - English Assistant Telegram Bot


*This notebook contains the documentation and the code for the Qalam Telegram bot project. Qalam is designed to help students improve their English language skills by providing grammar correction and word definitions. It uses the `python-telegram-bot` library along with `SpaCy` and `NLTK` for natural language processing.*

*In this notebook, we'll walk through the code and explain how it works step by step.*


**Setting Up Logging for the Bot**

*In this code, we set up logging to keep track of the bot's activities and debug information. Logging is crucial for troubleshooting and monitoring the bot's behavior during execution.*

In [1]:
import logging

# Create a logger
logger = logging.getLogger(__name__)

# Set up logging to both a file and the console
logger.setLevel(logging.DEBUG)

# Create handlers for file and console logging
file_handler = logging.FileHandler('bot.log')
console_handler = logging.StreamHandler()

# Create formatter and add it to handlers
formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')
file_handler.setFormatter(formatter)
console_handler.setFormatter(formatter)

# Add handlers to the logger
logger.addHandler(file_handler)
logger.addHandler(console_handler)

# Example log message
logger.debug("Bot has started")



2025-03-23 01:09:06,632 - DEBUG - Bot has started


**Importing Necessary Libraries**

*In this section, we import all the essential libraries required to run the Qalam Telegram bot. These libraries handle various tasks such as interacting with the Telegram API, processing natural language, and managing the bot's functionality. Below is a summary of the imported libraries:*

1. **os**: For interacting with the operating system, such as accessing environment variables.
2. **asyncio**: For asynchronous programming, enabling the bot to handle multiple tasks concurrently.
3. **nest_asyncio**: To allow nested use of asyncio, ensuring compatibility with Jupyter Notebooks.
4. **datetime**: To manage date and time operations.
5. **dotenv**: For loading environment variables from a `.env` file to securely store sensitive informati Telegram on like AI tokens.
6. **telegram**: The main library used for building and interacting with the Telegram bot.
7. **language_tool_python**: A library used to check grammar and spelling mistakes in sentences.
8. **nltk**: The Natural Language Toolkit (NLTK) used for text processing and analyzing words, sentences, and their structure.
9. **spacy**: A powerful library for advanced natural language processing.
10. **pytz**: For working with time zones.

*By importing these libraries, we can build a fully functional Telegram bot capable of correcting grammar, defining words, and providing additional language learning features.*



In [2]:
#import all necessary libraries 
import os
import asyncio
import nest_asyncio
from datetime import datetime
from dotenv import load_dotenv
from telegram import Update
from telegram.ext import Application, CommandHandler, MessageHandler, filters, ContextTypes, CallbackContext
from telegram.constants import ParseMode
import language_tool_python
import nltk
from nltk import pos_tag
from nltk.corpus import wordnet
from nltk.corpus import wordnet as wn
from nltk.corpus import stopwords
from nltk.corpus import words
from nltk.tokenize import word_tokenize
import pytz
from datetime import datetime
from telegram import Bot
import spacy
import signal
import sys

 **Download necessary NLTK datasets for NLP tasks**

 1. **punkt**: Used for tokenizing sentences and words.
 2. **wordnet**: A lexical database for English, useful for word meanings, synonyms, and antonyms.
 3. **omw-1.4**: Open Multilingual WordNet, which provides multilingual word mapping.
 4. **words**: A list of English words, useful for spell-checking or filtering non-dictionary words.

In [3]:
# Download necessary NLTK datasets for NLP tasks

nltk.download(['punkt', 'wordnet', 'omw-1.4', 'words'], quiet=True)


True

In [7]:
# Apply nest_asyncio to avoid event loop conflicts in Jupyter Notebook
nest_asyncio.apply()

In [8]:
# Load environment variables
load_dotenv()
TOKEN = os.getenv('TELEGRAM_TOKEN')
BOT_USERNAME = os.getenv('BOT_USERNAME')

**Loading and Processing Data**


*In this section, we load the required SpaCy model and NLTK resources needed for processing text.*

In [9]:
# Initialize LanguageTool for grammar checking
tool = language_tool_python.LanguageTool('en-US')


In [10]:
# Load the spaCy English language model
nlp = spacy.load('en_core_web_sm')

**Bot Commands and Functions**

*This section provides an overview of the key functions and commands used in the Qalam Telegram bot, along with their dependencies.*

### **Dependencies**
The following libraries and packages are used to implement the bot's functionality:

- **python-telegram-bot**: For interacting with the Telegram API.
- **language_tool_python**: For grammar checking and error correction.
- **nltk**: For natural language processing, including word definitions from WordNet.
- **spacy**: For advanced tokenization and lemmatization of words.

### **Commands**

1. **Start Command & Help Command**  
   The `/start` command welcomes users and provides an introduction to the bot's features.

   - **Function**: Sends a welcome message outlining available commands.
   
2. **Help Command**  
   The `/help` command explains how to use the bot and lists all available commands.

   - **Function**: Provides users with a guide on how to interact with the bot.

   **Screenshot of the bot's reply to the `/start` command and `/help` command:**
   <img src="images/start%20and%20help%20commands.png" alt="Start Command and Help Command" width="50%" style="display: block; margin: 0 auto;"/>

---

### **Functions**

1. **Correct Grammar Function**  
   The `/correct` command checks the grammar of the sentence provided by the user using the **language_tool_python** library. It suggests corrections if there are any errors in the sentence.

   - **Function**: Detects grammar mistakes and sends back the corrected version of the sentence.

   **Screenshot of the bot's reply to the `/correct` command:**
   <img src="images/correct%20.png" alt="Correct Grammar Function" width="50%" style="display: block; margin: 0 auto;"/>

2. **Define Word Function**  
   The `/define` command retrieves the definition, part of speech, and example usage of a word using the **nltk.wordnet** corpus and **spacy** for tokenization and lemmatization. SpaCy helps by processing the word, making the definition more precise.

   - **Function**: 
     - Fetches and displays the word's definition and usage examples from **WordNet**.
     - Uses **SpaCy** to process and lemmatize the word, and identify its part of speech, ensuring more accurate results and better handling of variations in word forms.

   **Screenshot of the bot's reply to the `/define` command:**
   <img src="images/define%20.png" alt="Define Function" width="50%" style="display: block; margin: 0 auto;"/>

*These functions are essential for providing language learning assistance and enhancing the user experience by offering grammar checks and word definitions directly through Telegram.*
checks and word definitions directly through Telegram.*


In [11]:
# Handle the /start command by sending a welcome message

async def start(update: Update, context: ContextTypes.DEFAULT_TYPE):
    logger.info(f"Received command: {update.message.text}")
    """Handle /start command"""
    await update.message.reply_text("""👋 Hello! I'm Qalam, your English assistant bot. I can:\n
📖 Correct grammar errors → Use /correct [sentence]\n
📚 Define words → Use /define [word]\n
🆘 Need help? Type /help
"""
       
    )

In [12]:
# Handle the /help command by providing usage instructions

async def help_command(update: Update, context: ContextTypes.DEFAULT_TYPE):
    logger.info(f"Received command: {update.message.text}")
    """Handle /help command"""
    help_text = (
        "ℹ️ **Qalam Bot Help**\n\n"
        "/start - Start conversation\n"
        "/correct <text> - Check grammar\n"
        "/define <word> - Get word definition, examples, and synonyms\n"
        "/help - Show this help message"
    )
    await update.message.reply_text(help_text, parse_mode=ParseMode.MARKDOWN)


In [14]:
# Handle the /correct command to identify and suggest corrections for grammar mistakes

async def correct_grammar(update: Update, context: CallbackContext):
    """Handle the /correct command by correcting grammar and giving feedback."""
    if not context.args:
        await update.message.reply_text("Please provide a sentence: /correct [your sentence]")
        return

    # Get the user's input sentence
    user_sentence = " ".join(context.args)

    # Analyze grammar mistakes
    matches = tool.check(user_sentence)
    corrected_sentence = tool.correct(user_sentence)

    # Construct response message
    response = "📝 **Grammar Check Report:**\n\n"

    if not matches:
        response += "✅ No grammar mistakes found!\n\n"
    else:
        for match in matches:
            incorrect_text = match.context  # Context of the incorrect phrase
            suggestion = match.replacements[0] if match.replacements else "No suggestion available"
            
            response += f"🔸 **Error:** {match.message}\n\n"
            response += f"  ✏️ **Incorrect Phrase:** `{incorrect_text}`\n"
            response += f"  ✅ **Suggested Correction:** `{suggestion}`\n\n"

    # Send the corrected sentence
    response += f"✅ **Final Correction:** `{corrected_sentence}`"

    await update.message.reply_text(response, parse_mode="Markdown")

In [15]:
# Handle the /define command to provide word definitions,synonyms, POS tagging and examples

async def define_word(update: Update, context: ContextTypes.DEFAULT_TYPE):
    logger.info(f"Received command: {update.message.text}")
    """Handle /define command with synonyms and POS tagging"""
    try:
        word = ' '.join(context.args).lower()
        if not word:
            await update.message.reply_text("Please provide a word after /define")
            return

        # Get WordNet synsets
        synsets = wordnet.synsets(word)
        if not synsets:
            await update.message.reply_text(f"❌ No definition found for '{word}'")
            return

        # Use spaCy for POS tagging
        doc = nlp(word)
        pos_tag = doc[0].pos_

        # Start forming the response
        response = f"📚 **{word.capitalize()}**\n\n"
        response += f"🔸 *Part of Speech:* {pos_tag}\n\n"
       
        # Get the first three synsets (different meanings)
        for i, synset in enumerate(synsets[:3], 1):
            # Definition
            definition = synset.definition()
            response += f"**Meaning {i}:** {definition}\n"
           
            # Examples
            examples = synset.examples()[:2]
            if examples:
                response += f"🔹 *Examples:*\n- " + "\n- ".join(examples) + "\n"
           
            # Synonyms handling
            lemmas = [lemma.name().replace('_', ' ') for lemma in synset.lemmas()]
            unique_synonyms = list(set([lem for lem in lemmas if lem != word]))
           
            if unique_synonyms:
                response += f"🔸 *Synonyms:* {', '.join(unique_synonyms)}\n\n"
            else:
                response += "🔸 *No distinct synonyms found*\n\n"
       
        # Send the response
        await update.message.reply_text(response, parse_mode=ParseMode.MARKDOWN)
       
    except Exception as e:
        logger.error(f"Word definition error: {e}")
        await update.message.reply_text("⚠️ Error fetching word information")

In [16]:
# Handle text messages that are not commands

async def handle_message(update: Update, context: ContextTypes.DEFAULT_TYPE):
    user_message = update.message.text.lower()  # Convert to lowercase for consistency
    
    if "check grammar" in user_message or "correct this" in user_message:
        response = check_grammar(user_message)  # Call your grammar function
        await update.message.reply_text(response)
    else:
        await update.message.reply_text("I'm not sure, sorry...")


In [17]:
# Handle errors that occur during the bot's execution

async def error_handler(update, context):
    try:
        if update and update.message:
            print(f"Update {update} caused error: {context.error}")
        else:
            print(f"An error occurred: {context.error}")
    except Exception as e:
        print(f"Error handling the error: {e}")


In [18]:
# This function handles incoming messages based on whether the bot is mentioned in a group chat or if it's a direct message.

async def process_message(message_type: str, text: str, BOT_USERNAME: str) -> str:
    if message_type == 'group':
        if BOT_USERNAME in text:
            new_text = text.replace(BOT_USERNAME, '').strip()
            response = await handle_response(new_text)  # Await handle_response
        else:
            return ''  # Return empty if no bot mention
    else:
        response = await handle_response(text)  # Await handle_response for normal messages

    return response

In [19]:
# Set bot's timezone
timezone = pytz.timezone('Asia/Riyadh')
dt = datetime.now(timezone)

## **Main Function: Bot Initialization and Execution**

*This section explains the main function of the Qalam Telegram bot. It serves as the entry point to start and run the bot, handling the bot's lifecycle, including command handling, error management, and logging.*

### **Function Overview**
The `main` function is the core function that runs the bot. It initializes the bot, sets up command and message handlers, and starts the bot's polling mechanism to listen for updates. It also manages error handling and logging.


In [20]:
# main function
# This function serves as the entry point for running the bot. 

async def main():
    """Start the bot"""
    try:
        # Initialize the bot
        app = Application.builder().token(TOKEN).build()

        # Command handlers
        app.add_handler(CommandHandler("start", start))
        app.add_handler(CommandHandler("correct", correct_grammar))
        app.add_handler(CommandHandler("define", define_word))
        app.add_handler(CommandHandler("help", help_command))
        
        # Message handler
        app.add_handler(MessageHandler(filters.TEXT & ~filters.COMMAND, handle_message))
        
        # Error handler
        app.add_error_handler(error_handler)

        # Inform that the bot is running
        print("bot is running..")
        logger.info("Starting Qalam bot...")
        
        # Run the bot
        await app.run_polling(drop_pending_updates=True)
    
    except Exception as e:
        logger.error(f"Error in starting bot: {e}")

# Entry point to run the bot
if __name__ == "__main__":
    try:
        loop = asyncio.get_event_loop()
        if loop.is_running():
            loop.create_task(main())  # Works in Jupyter Notebook
        else:
            loop.run_until_complete(main())  # Standard script execution
    except KeyboardInterrupt:
        print("Shutting down the bot...")

2025-03-23 01:09:19,432 - INFO - Starting Qalam bot...
Starting Qalam bot...


bot is running..
An error occurred: Conflict: terminated by other getUpdates request; make sure that only one bot instance is running
An error occurred: Conflict: terminated by other getUpdates request; make sure that only one bot instance is running
An error occurred: Conflict: terminated by other getUpdates request; make sure that only one bot instance is running
An error occurred: Conflict: terminated by other getUpdates request; make sure that only one bot instance is running
An error occurred: Conflict: terminated by other getUpdates request; make sure that only one bot instance is running
An error occurred: Conflict: terminated by other getUpdates request; make sure that only one bot instance is running
An error occurred: Conflict: terminated by other getUpdates request; make sure that only one bot instance is running
An error occurred: Conflict: terminated by other getUpdates request; make sure that only one bot instance is running



___
## **Future Improvements**

- *Multilingual Support*: *We plan to add support for multiple languages, including Arabic, in future versions.*  
- *Advanced Grammar Checking*: *Integrate more sophisticated grammar-checking models or APIs for better accuracy.*  
- *User Feedback*: *Implement a feedback system to help improve the bot's suggestions over time.*


## **Conclusion**

*This notebook provides an overview of the Qalam Telegram bot project. The bot leverages natural language processing techniques to help students with grammar and vocabulary. By following the steps outlined in this notebook, you can run and customize the bot to fit your own needs.*  

*Feel free to reach out with any questions or suggestions!*
